In [9]:
from database.connect import getConnection
import JsonHandler as jh
import mysql
import pandas as pd
from pathlib import Path
import csv

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [6]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

In [7]:
print(jh.languages_list)


NameError: name 'jh' is not defined

In [7]:
airlines_dict = {'KLM': ['klm'],
                'AirFrance':['airfrance',
                            'air france'],
                'British_Airways': ['british_airways',
                                    'british airways'],
                'AmericanAir': ['americanair',
                                'american airlines'],
                'Lufthansa': ['lufthansa'],
                'AirBerlin': ['airberlin',
                                'air berlin'],
                'AirBerlin assist': ['airberlin assist',
                                    'air berlin assist',
                                    'airberlinassist'],
                'easyJet': ['easyjet'],
                'RyanAir': ['ryanair'],
                'SingaporeAir': ['singaporeair',
                                'singapore airlines'],
                'Qantas': ['qantas'],
                'EtihadAirways': ['etihad airways',
                                'etihadairways',
                                'etihad'],
                'VirginAtlantic': ['virgin atlantic',
                                    'virginatlantic'],
            }

In [8]:
# Create a cursor to execute SQL queries
cursor = connection.cursor()

# open the csv file
with open(Path('output/airline_racism.csv'), 'w') as f:
    f.write("Airline,Language,ParticipatedPercentage,TotalConversations,ParticipatedConversations\n")
    for key, value in airlines_dict.items():
        query = f"""
            SELECT
                t.language AS Language,
                (SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS ParticipatedPercentage,
                COUNT(*) AS TotalConversations,
                SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) AS ParticipatedConversations
            FROM
                conversations c
                INNER JOIN part_of p ON c.id = p.cID
                INNER JOIN tweets t ON t.id = p.tID
            WHERE
                (t.airlines LIKE '%lazy_query_strat_ignore_this%'
        """
        for airline in value:
            query += f" OR t.airlines LIKE '%{airline}%'"

        query += """
            )
            GROUP BY Language
        """

        cursor.execute(query)
        result = cursor.fetchall()
        # save the results to a csv file
        airline = key
        print(airline)
        for row in result:
            language = row[0]
            participated_percentage = row[1]
            total_conversations = row[2]
            participated_conversations = row[3]
            f.write(f"{airline},{language},{participated_percentage},{total_conversations},{participated_conversations}\n")




In [9]:
# create the dataframe to store the language counts
df = pd.DataFrame(columns=['language', 'sentiment'])

# get all the sentiment scores per language
for language in jh.languages_list:
    query = """
    SELECT sentiment
    FROM tweets
    WHERE language = %s
    """
    cursor.execute(query, (language,))
    sentiments = cursor.fetchall()
    sentiment_list = [sentiment[0] for sentiment in sentiments]

    # add the language and the sentiment to the dataframe
    df.loc[len(df.index)] = [language, sentiment_list]

# save the dataframe to a csv file
df.to_csv(Path('output/sentiment_per_language.csv'), index=False)

In [ ]:
# Close the cursor and connection
cursor.close()
connection.close()
